In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

from dotenv import load_dotenv
import sys
import os
from pathlib import Path
load_dotenv()
# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config_loader import config
from src.utils import anthropic_generate, load_json, extract_json_from_llm_response, save_json, list_story_folders
from src.chat import get_challenge_generation_prompt, create_html_challenges, get_html_challenge_inputs

Default FFmpeg path added to system PATH: C:\Program Files\ffmpeg-7.0-essentials_build\bin


In [14]:
from src.gcs_storage import get_stories_from_collection, get_story_dialogue_path, get_story_challenges_path, upload_to_gcs, read_from_gcs, get_story_translated_challenges_path

all_stories = get_stories_from_collection(collection="LM1000")

## Generate scenarios

In [11]:
story_dialogue = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=get_story_dialogue_path(story_name="story_midnight_garden_mystery", collection="LM1000"))

In [13]:
prompt = get_challenge_generation_prompt(story_dialogue)
scenario_dicts = extract_json_from_llm_response(anthropic_generate(prompt, max_tokens=5000))

upload_to_gcs(obj=scenario_dicts, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=get_story_challenges_path(story_name="story_midnight_garden_mystery", collection="LM1000"))

Function that called this one: <module>. Sleeping for 20 seconds


'gs://audio-language-trainer-private-content/collections/LM1000/stories/story_midnight_garden_mystery/challenges.json'

## If scenarios already exist start here

In [21]:
scenario_dicts = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=get_story_challenges_path(story_name="story_midnight_garden_mystery", collection="LM1000"))
challenges = get_html_challenge_inputs(scenario_dicts) # this is the first time config.TARGET_LANGUAGE_NAME is used
chat_webpage_file = create_html_challenges(challenges, story_name="story_midnight_garden_mystery")

HTML challenges created at: gs://audio-language-trainer-stories/french/story_midnight_garden_mystery/challenges.html


In [19]:
upload_to_gcs(file_path=chat_webpage_file, bucket_prefix=f"{config.TARGET_LANGUAGE_NAME}/{STORY_NAME}")

'https://storage.googleapis.com/audio-language-trainer-stories/swedish/story_community_park/challenges.html'